In [ ]:
import torch
import torch.nn as nn
import pandas as pd

In [ ]:
!unzip hate_speech.zip

Archive:  hate_speech.zip
  inflating: labeled_data.csv        


In [ ]:
!pip install opendatasets


In [ ]:
import opendatasets as od
od.download("https://www.kaggle.com/datasets/rahulgoel1106/xenophobia-on-twitter-during-covid19")

Skipping, found downloaded files in "./xenophobia-on-twitter-during-covid19" (use force=True to force download)


In [ ]:
xen = pd.read_csv("xenophobia-on-twitter-during-covid19/Xenophobia.csv", encoding="latin1")

ParserError: Error tokenizing data. C error: EOF inside string starting at row 4722520

In [ ]:
df = pd.read_csv("labeled_data.csv")

In [ ]:
xen["text"] = xen["text"].astype(str)

In [ ]:
import re

mention_re = '@\w+\s*'
url_re = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
num_pattern = '[0-9]'


def remove_mentions(tweet):
    return re.sub(mention_re, '', tweet)


def remove_urls(tweet):
    return re.sub(url_re, '', tweet)

def remove_nums(tweet):
    return re.sub(num_pattern, '', tweet)


df["tweet"] = df["tweet"].apply(remove_mentions)
df["tweet"] = df["tweet"].apply(remove_urls)
df["tweet"] = df["tweet"].apply(remove_nums)
df["tweet"] = df["tweet"].apply(lambda x: x.replace("RT", ""))


xen["text"] = xen["text"].apply(remove_mentions)
xen["text"] = xen["text"].apply(remove_urls)
xen["text"] = xen["text"].apply(remove_nums)
xen["text"] = xen["text"].apply(lambda x: x.replace("RT", ""))


In [ ]:
from string import punctuation
# remove puncs
for elm in punctuation:
  df["tweet"] = df["tweet"].apply(lambda x: x.replace(elm, ""))
  xen["text"] = xen["text"].apply(lambda x: x.replace(elm, ""))

In [ ]:
df["tweet"] = df["tweet"].str.lower()
xen["text"] = xen["text"].str.lower()


In [ ]:
hate_only = df[df.hate_speech == 1]



In [ ]:
def features_to_dataframe(corpus, vectorizer):
    X = vectorizer.fit_transform(corpus)
    features = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())
    return features

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

corpus = hate_only.tweet
count_vectorizer = CountVectorizer()
tf_vectorizer = TfidfVectorizer(stop_words="english")



(features_to_dataframe(corpus, tf_vectorizer).sum()).sort_values(ascending=False).head(50)

bitch       180.594286
nigga        88.703394
bitches      83.219845
hoes         78.967750
like         72.893048
fuck         72.034183
ass          71.144462
hoe          68.887061
niggas       67.748085
pussy        66.563524
dont         60.894860
im           54.373915
faggot       49.052100
just         47.647523
got          45.969651
shit         44.607426
aint         41.350593
fucking      38.697642
lol          36.048285
trash        35.867536
youre        34.264163
know         33.369191
fag          32.672117
nigger       30.752083
yall         29.844019
niggah       28.585294
hate         27.700358
amp          27.666087
yo           27.453307
want         26.463401
real         24.996637
white        24.859819
love         24.717766
man          23.391487
thats        23.085216
dumb         22.961096
cunt         22.121417
ya           20.532675
look         20.059738
retarded     20.030425
say          19.619528
stupid       19.466040
retard       19.184249
lmao       

In [ ]:
tf_idf = (features_to_dataframe(corpus, tf_vectorizer).sum())

tf_idf.sort_values(ascending=False).tail(60)


amend            0.266320
purpose          0.266320
das              0.261296
gero             0.261296
thanx            0.260053
grease           0.260053
bona             0.258790
resort           0.258790
qaeda            0.258790
reclaim          0.258790
pressure         0.258790
fides            0.258790
disturbing       0.258790
uturn            0.253931
instancenikki    0.253931
shooters         0.253916
boit             0.252097
wings            0.247811
throws           0.247811
grabs            0.247811
stops            0.247811
tol              0.246782
caddy            0.246782
station          0.246782
trued            0.246782
ons              0.246782
prick            0.246069
whippin          0.246069
oath             0.246069
routed           0.246069
vp               0.246069
breaking         0.246069
barrybonds       0.245598
steroid          0.245598
ohhgt            0.245598
forgiving        0.245598
giantsgt         0.245598
busterposey      0.245598
kdot        

In [ ]:

racial_keywords = ["nigga", "niggas", "nigger",
                   "niggah", "white", "black","nigguh" ,
                   "niggers", "niggaz", "wigga"]


xenophobic_keywords = ["we need to enforce our borders", "learn to speak english",
                "are terrorists","where are you really from", "are spreading virus",
                "i’m not like other", "illegal alien", "illegal immigrant",
                    "illegal immigration", "send her back", "send them back", "illegal criminal"]




hate_only[hate_only.tweet.str.contains("illegal")]


,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
3352,3438,3,1,0,2,2,ill ask you same question do you support borde...
9924,10195,3,1,0,2,2,how long after the redskins change their nsme ...
15086,15443,3,1,0,2,2,illegals bringing diseases across border con...
17127,17518,3,1,2,0,1,obama is allowing millions of illegal aliens...


In [ ]:
tf_idf[tf_idf<8].sort_values(ascending = False).head(50)

kids        7.990979
work        7.958450
niggaz      7.953581
yea         7.949241
feel        7.920376
straight    7.913479
nig         7.912493
looking     7.870536
head        7.819775
cut         7.818839
bruh        7.817152
friends     7.682084
big         7.642217
nah         7.527308
face        7.488247
went        7.480006
baby        7.456850
today       7.454404
school      7.440316
young       7.417973
getting     7.410789
tf          7.353055
days        7.332927
pussies     7.299130
hell        7.217726
fight       7.155582
gone        7.151767
stand       7.142697
called      7.137254
watch       7.096559
dude        7.084963
yes         7.080790
dis         7.076069
fake        7.053796
ive         7.044161
told        7.039596
coon        6.980591
funny       6.926139
guys        6.902768
start       6.797440
haha        6.796077
text        6.747116
night       6.742238
wont        6.709798
wow         6.664968
kill        6.661561
happy       6.650949
having      6

In [ ]:
def xenophobic_match(tweet, search_term):
      for s in search_term:
        if s in tweet.lower():
            return True
      return False



In [ ]:
xen["target"]=xen.apply(lambda x : xenophobic_match(x.text, xenophobic_keywords), axis=1)
xen['target'].value_counts()

target
False    4767571
True         838
Name: count, dtype: int64

In [ ]:
hate_only["target"] = hate_only.apply(lambda x : xenophobic_match(x.tweet, racial_keywords), axis = 1)
hate_only["target"].value_counts()

<ipython-input-15-9550a7333dbe>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hate_only["target"] = hate_only.apply(lambda x : xenophobic_match(x.tweet, racial_keywords), axis = 1)


target
False    2372
True     1047
Name: count, dtype: int64

In [ ]:
# hate_only[hate_only["target"] == True]
hate_only[hate_only["target"] == True]


,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet,target
49,49,3,1,2,0,1,these hoes like niggas that spend money not t...,True
50,50,3,1,2,0,1,we dont trust these niggas all these bitches,True
69,69,3,1,2,0,1,fuck red malone man bitch ass niggah could yo...,True
77,77,3,1,2,0,1,u ever kill a ant on the sidewalk and think d...,True
79,79,3,1,2,0,1,ppl talk bad about the ghettohood but as a k...,True
...,...,...,...,...,...,...,...,...
24731,25244,3,1,2,0,1,yall niggers have a good thanksgiving,True
24732,25245,3,1,2,0,1,yal some faggots yup nigga you n you fuckin fr...,True
24752,25265,6,1,5,0,1,you aint a real nigga without goal a to chase ...,True
24762,25275,3,1,2,0,1,you got niggas and i got bitches,True


In [ ]:
def clean_text(text):
    text = text.encode('ascii', 'ignore').decode('ascii')
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [ ]:
hate_only["tweet"] = hate_only["tweet"].apply(clean_text)

<ipython-input-18-2a1a3c340166>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hate_only["tweet"] = hate_only["tweet"].apply(clean_text)


In [ ]:
hate_only

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet,target
5,5,3,1,2,0,1,the shit just blows meclaim you so faithful an...,False
9,9,3,1,2,0,1,hobbies include fighting mariam bitch,False
14,14,3,1,2,0,1,bitch get up off me,False
17,17,3,1,2,0,1,bitch who do you love,False
49,49,3,1,2,0,1,these hoes like niggas that spend money not ta...,True
...,...,...,...,...,...,...,...,...
24749,25262,3,1,2,0,1,yo hoe will get slayed,False
24752,25265,6,1,5,0,1,you aint a real nigga without goal a to chase ...,True
24753,25266,3,1,2,0,1,you aint gotta be a dyke to like hoes,False
24762,25275,3,1,2,0,1,you got niggas and i got bitches,True


In [ ]:
xen["text"] = xen["text"].apply(clean_text)

In [ ]:
xen_text = xen[xen["target"] == True]["text"]
race_text = hate_only[hate_only["target"] == True]["tweet"]
race_xen = pd.concat([xen_text, race_text])
race_xen = race_xen.reset_index()
race_xen.pop("index")
race_xen

,0
0,does not register the victims to avoid convict...
1,million americans dont have health insurance w...
2,reassuring to know that all testing done for h...
3,illegal immigrants forbidden whats the problem...
4,i know were rightly trying to limit the politi...
...,...
1880,yall niggers have a good thanksgiving
1881,yal some faggots yup nigga you n you fuckin fr...
1882,you aint a real nigga without goal a to chase ...
1883,you got niggas and i got bitches


In [ ]:
race_xen["label"] = 1
race_xen.rename(columns={0: 'tweet'}, inplace=True)

race_xen # xen and race text are defined

,tweet,label
0,does not register the victims to avoid convict...,1
1,million americans dont have health insurance w...,1
2,reassuring to know that all testing done for h...,1
3,illegal immigrants forbidden whats the problem...,1
4,i know were rightly trying to limit the politi...,1
...,...,...
1880,yall niggers have a good thanksgiving,1
1881,yal some faggots yup nigga you n you fuckin fr...,1
1882,you aint a real nigga without goal a to chase ...,1
1883,you got niggas and i got bitches,1


In [ ]:
non_hate = df[df["hate_speech"] == 0]["tweet"].reset_index()

non_hate.pop("index")
non_hate["label"] = 0
non_hate



,tweet,label
0,as a woman you shouldnt complain about clea...,0
1,boy dats coldtyga dwn bad for cuffin dat ho...,0
2,dawg you ever fuck a bitch and she start t...,0
3,she look like a tranny,0
4,the shit you hear about me might be true or...,0
...,...,...
19785,yous a muthafin lie right his tl is trash no...,0
19786,youve gone and broke the wrong heart baby and ...,0
19787,young buck wanna eat dat nigguh like i aint fu...,0
19788,youu got wild bitches tellin you lies,0


In [ ]:
dataset = pd.concat([non_hate, race_xen])
dataset = dataset.reset_index()
dataset.pop("index")
dataset

,tweet,label
0,as a woman you shouldnt complain about clea...,0
1,boy dats coldtyga dwn bad for cuffin dat ho...,0
2,dawg you ever fuck a bitch and she start t...,0
3,she look like a tranny,0
4,the shit you hear about me might be true or...,0
...,...,...
21670,yall niggers have a good thanksgiving,1
21671,yal some faggots yup nigga you n you fuckin fr...,1
21672,you aint a real nigga without goal a to chase ...,1
21673,you got niggas and i got bitches,1


In [ ]:
dataset['text_length'] = dataset['tweet'].str.len()
 # 68


0    44
Name: text_length, dtype: int64

In [ ]:

dataset.to_csv('xenophobic_data.csv', index=False)
